# 量子フーリエ変換

古典的なフーリエ変換は波や信号の解析において重要なツールであり、関数をそれぞれ異なる周波数を持つ成分に分解します。

この離散的な対応である離散フーリエ変換は、$n$ 個の複素数 $x_0,\ldots,x_{N-1}$ に作用し、次式のように、別の $n$ 個の複素数列 $\tilde x_0,\ldots,\tilde x_{N-1}$ へと変換します。

$$\tilde x_k = \sum_{y=0}^{N-1}e^{-\frac{2\pi ikn}N} \cdot x_k$$

$n$ 個の量子ビットに対する量子フーリエ変換（一般にQFTと略します）は各基底状態 $x\in \{0,1\}^n$ に対して同様にはたらき、次式のように表されます（ただし、$N=2^n$）。

$$\text{QFT}(\lvert x\rangle) = \frac 1{\sqrt N}\sum_{y=0}^{N-1}e^{\frac{2\pi ixy}N}\lvert y\rangle\qquad \qquad (1)$$

ここでは、ビット列 $x\in\{0, 1\}^n$ を整数 $\sum_{j=0}^{n-1}2^{n-1-j}x_j$ として表記するものとします。

QFTは事実上、計算基底から "フーリエ基底" に基底を変換したものと考えることができます。

## QFTのテンソル積分解

いくつかの代数的な操作によって、式$(1)$の総和を純粋状態同士のテンソル積へと分解できることがわかります。

$$\text{QFT}(\lvert x\rangle) = \frac 1{\sqrt N}\left(\vert 0\rangle + e^{\frac{2\pi ix}{2^1}}\vert 1\rangle\right)\otimes \left(\vert 0\rangle + e^{\frac{2\pi ix}{2^2}}\vert 1\rangle\right)\otimes \cdots \otimes \left(\vert 0\rangle + e^{\frac{2\pi ix}{2^n}}\vert 1\rangle\right).$$

この定式化により、QFTを実現する量子回路をどのように実装すればよいかが明確になります。

## 制御位相回転

回路構成を見る前に、次のゲートを定義します。

$$R_k = \begin{pmatrix}1 & 0\\ 0 & e^{\frac{2\pi i}{2^k}}\end{pmatrix},$$

そしてこのゲートを制御ゲート化したものが、制御量子ビットの値に応じてターゲット量子ビットの位相を変えることを思い出しましょう。  
言い換えると、$CR_k(\lvert 0\rangle|x\rangle) = |0\rangle \lvert x\rangle$ かつ

\begin{aligned}CR_k(\lvert 1\rangle \lvert x\rangle) = \lvert 1\rangle\otimes R_k(\lvert x \rangle) &=\lvert 1\rangle\otimes R_k\left( \langle 0\lvert x\rangle \lvert  0\rangle + \langle 1 \lvert x\rangle \lvert 1\rangle\right)
 \\ & = \lvert 1\rangle\otimes \left( \langle 0 \lvert x\rangle \lvert  0\rangle + \langle 1 \lvert x\rangle e^{\frac{2\pi i}{2^k}} \lvert 1\rangle\right)\end{aligned}
 
 となります。

より簡潔に、次のように表されます。

$$CR_k(\lvert y\rangle \lvert x\rangle) = \lvert y\rangle \otimes \left( \langle 0 \vert x\rangle \lvert  0\rangle + \langle 1 \vert x\rangle e^{\frac{2\pi iy}{2^k}} \lvert 1\rangle\right).$$

よって、  
$$CR_k = \begin{pmatrix}1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0\\ 0& 0& 1 & 0\\ 0 & 0&0 &e^{\frac{2\pi i}{2^k}}\end{pmatrix} = \text{CPhase}\left(\frac{2\pi}{2^k}\right).$$

## 回路構造

これで、QFTを実現する回路を構築するために必要な材料が揃いました。  
以下は、$n=4$量子ビットの回路例です。

![Circuit Image](../tutorial/img/121_qft_ckt.png)

ここでは、$\lvert \varphi_1\rangle$から$\lvert \varphi_4\rangle$までの状態を見ていきます。

$\lvert \varphi_1\rangle$ において、先頭の量子ビットの状態を、基底状態 $x_0\in\{0,1\}$ 上で解析します。すると、$\lvert x_0\rangle$ は次のようになります（ここでは計算を簡単にするために、正規化は無視します）。
+ $H$: $\lvert 0\rangle + e^{\pi i x_0}\lvert 1\rangle$,
+ $R_2$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2}}\lvert 1\rangle$
+ $R_3$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2} + \frac{2\pi i x_2}{2^3}}\lvert 1\rangle$
+ $R_4$: $\lvert 0\rangle + e^{\pi i x_0 + \frac{2\pi i x_1}{2^2} + \frac{2\pi i x_2}{2^3} + \frac{2\pi i x_3}{2^4}}\lvert 1\rangle = \lvert 0\rangle + e^{\frac{2\pi i}{2^4} (8x_0 + 4x_1 + 2x_2 + x_3)}\lvert 1\rangle = |0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle$.

よって、$\lvert \varphi_1\rangle = \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle\right)\otimes \lvert x_1x_2x_3\rangle$ となります。

帰納的に、$\lvert \varphi_2\rangle$ が量子ビット $\lvert x_1\rangle$ の状態を次のように変えることが簡単に確かめられます。
+ $\lvert 0\rangle + e^{\frac{2\pi i}{2^3}(4x_1 + 2x_2 + x_3)}\lvert 1\rangle$.  

ただし、$e^{2\pi ix_0} = 1$ なので、$e^{\frac{2\pi i}{2^3}(4x_1 + 2x_2 + x_3)}=e^{\frac{2\pi i}{2^3}(8x_0 + 4x_1 + 2x_2 + x_3)}= e^{\frac{2\pi ix}{2^3}}$ とできます。

よって $\lvert \varphi_2\rangle = \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^4}}\lvert 1\rangle\right)\otimes \left(\lvert 0\rangle + e^{\frac{2\pi i x}{2^3}}\lvert 1\rangle\right)\otimes \lvert x_2x_3\rangle$ となります。

このように続けると、 $\lvert \varphi_4 \rangle$ は私達が求める量子ビットを逆の順序で与えることが確認できます。そのため2つのスワップ操作を行って、QFTの回路を完成させました。

## Blueqat によるQFTの実装

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [1]:
# モジュールをインポート
from blueqat import Circuit
import math

In [2]:
# qftを作用させる関数
def apply_qft(circuit: Circuit(), qubits):
    num_qubits = len(qubits)
    for i in range(num_qubits):
        circuit.h[qubits[i]]
        for j in range(i+1, num_qubits):
            circuit.cphase(math.pi/(2 ** (j-i)))[qubits[j],qubits[i]] # Apply gate CR_{j-i}(qubit j, qubit i)
    # 最後に量子ビットの順序を逆にする
    for i in range(int(num_qubits/2)):
        circuit.swap(qubits[i],qubits[num_qubits-i-1])

### 回路の実行

In [3]:
n = 4  # QFTを行う量子ビット数
qc = Circuit()
qc.x[:]  # 状態 |1111> を用意
apply_qft(qc, range(n))
qc.run()

array([ 2.50000000e-01+0.j        ,  2.30969883e-01-0.09567086j,
        1.76776695e-01-0.1767767j ,  9.56708581e-02-0.23096988j,
       -1.53080850e-17-0.25j      , -9.56708581e-02-0.23096988j,
       -1.76776695e-01-0.1767767j , -2.30969883e-01-0.09567086j,
       -2.50000000e-01+0.j        , -2.30969883e-01+0.09567086j,
       -1.76776695e-01+0.1767767j , -9.56708581e-02+0.23096988j,
        1.53080850e-17+0.25j      ,  9.56708581e-02+0.23096988j,
        1.76776695e-01+0.1767767j ,  2.30969883e-01+0.09567086j])

QFTは個々の量子ビットの位相を変化させるだけなので、測定によって物理的に観測することはできません。以下は、$\text{QFT}(|1111\rangle)$から得られた上記の状態ベクトルを視覚的に表したものです。

![Statevector](../tutorial/img/121_qft_statevec.png)


## QFTの応用

QFT has wide applications in Algorithms design, and is the building block for many influential quantum algorithms such as Quantum Phase Estimation [[1]] , Shor's Algorithm [[2]], and the Hidden Subgroup Problem [[3]].

[1]: https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm
[2]: https://en.wikipedia.org/wiki/Shor's_algorithm
[3]: https://en.wikipedia.org/wiki/Hidden_subgroup_problem